<a href="https://colab.research.google.com/github/Mazen030/Movie-Recommendation-system/blob/main/Movie_Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import difflib #match the words to get value
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
movies_data=pd.read_csv('/content/movies.csv')

In [5]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
movies_data.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [7]:
 movies_data.shape

(4803, 24)

In [8]:
#features selection
selected_features=['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [9]:
#checking the null values of the selected columns
for feature in selected_features:
  movies_data[feature]=movies_data[feature].fillna('')


In [10]:
 #combining the features
 combined_features=movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [11]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [12]:
#convert textual data to numerical data to use cosine similarity
vectorizer=TfidfVectorizer()


In [13]:
feature_vector=vectorizer.fit_transform(combined_features)


In [14]:
print(feature_vector)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

Cosine similarity

In [15]:
similarity=cosine_similarity(feature_vector) #cosine function ta5od data numerical de w tdy kol wahed fehom score w elsimilar bya5do nfs elscore

In [ ]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [16]:
print(similarity.shape)

(4803, 4803)


In [17]:
#getting the movie name from the user
movie_name=input('enter your favourite movie name:')

enter your favourite movie name:scarface


In [19]:
#create a list with all the movies name given in the dataset
list_of_movies=movies_data['title'].tolist() #tolist btcreate list
print(list_of_movies)


['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [20]:
#finding the close the match given by the user
find_close_match=difflib.get_close_matches(movie_name,list_of_movies)
print(find_close_match)

['Scarface']


In [21]:
close_match=find_close_match[0]
print(close_match)

Scarface


In [22]:
#finding the index of movie based on the title
index_of_movie=movies_data[movies_data.title==close_match]['index'].values[0]
print(index_of_movie)

1850


In [23]:
#getting a list of similar movies based on similarity score
similarity_score=list(enumerate(similarity[index_of_movie]))
print(similarity_score)

[(0, 0.056610752035838104), (1, 0.034877255731489955), (2, 0.016269156112760638), (3, 0.04472812373843775), (4, 0.023329541238043587), (5, 0.01230514720097093), (6, 0.0), (7, 0.021739213565180496), (8, 0.0), (9, 0.006453982106706346), (10, 0.03294717683660266), (11, 0.01954963664347116), (12, 0.017685792238201804), (13, 0.010950356865093313), (14, 0.02910150055257184), (15, 0.0), (16, 0.020613395563247254), (17, 0.006392454513199789), (18, 0.011348922910398065), (19, 0.043540214938119805), (20, 0.011829152170364319), (21, 0.005492516578244566), (22, 0.004817346650546299), (23, 0.00922445907166103), (24, 0.03583735969890843), (25, 0.009531133124645496), (26, 0.022436758978469497), (27, 0.01638235909907654), (28, 0.02876371823944718), (29, 0.013235265942250033), (30, 0.006595962843830339), (31, 0.03164951406207466), (32, 0.0), (33, 0.012689150456098482), (34, 0.0), (35, 0.01823923259280238), (36, 0.016435004089941554), (37, 0.040055355972877155), (38, 0.006679237630144184), (39, 0.011186

In [24]:
#sorting the movies based on similarity score
sorted_movies=sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(sorted_movies)

[(1850, 1.0000000000000002), (2932, 0.28590547265382105), (1828, 0.18525402974399033), (3225, 0.17797557944212303), (2731, 0.17422901180662645), (186, 0.15792733624322589), (1792, 0.1570906127859189), (733, 0.14886515123586785), (428, 0.14769905725561866), (761, 0.1470572322996394), (1772, 0.14139783373052883), (398, 0.1324220363990092), (425, 0.13221764066202465), (591, 0.12612653386450629), (2280, 0.1257967119829596), (2985, 0.1255036408720826), (2649, 0.12010108355712529), (587, 0.11893846727898898), (3337, 0.11790399789345675), (901, 0.11717540794045055), (2543, 0.11678398760107274), (1394, 0.11634288592621696), (1024, 0.11630662336572066), (3050, 0.11621440717612047), (3423, 0.11611086636424661), (2240, 0.1155720833379345), (1458, 0.11316155519870823), (1250, 0.11217910306909476), (3637, 0.11118472073144448), (4097, 0.11100557849983593), (373, 0.11015836035703652), (3654, 0.11008288120644964), (4318, 0.10878969791154035), (1033, 0.10866529730566954), (584, 0.10781543178429873), (2

In [25]:
#print the name of similar movies based on the index
print('movies suggested for you:')
i=1
for movie in sorted_movies:
  index=movie[0]
  title_from_index=movies_data[movies_data.index==index]['title'].values[0]
  if(i<30):
    print(i,'.',title_from_index)
    i+=1

movies suggested for you:
1 . Scarface
2 . Raising Cain
3 . The Untouchables
4 . Limbo
5 . The Godfather: Part II
6 . Bad Boys II
7 . Pain & Gain
8 . Up Close & Personal
9 . Batman Returns
10 . Righteous Kill
11 . Flawless
12 . Ocean's Thirteen
13 . Mission: Impossible
14 . Stardust
15 . Sea of Love
16 . The Color of Money
17 . The Son of No One
18 . The Abyss
19 . The Godfather
20 . Robin Hood: Prince of Thieves
21 . Cop Land
22 . Donnie Brasco
23 . Dick Tracy
24 . Body Double
25 . Dressed to Kill
26 . A Most Violent Year
27 . Femme Fatale
28 . City Hall
29 . Kill the Messenger


In [27]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_movie]))

sorted_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : godfather
Movies suggested for you : 

1 . The Godfather
2 . The Godfather: Part III
3 . Apocalypse Now
4 . Closer
5 . The Godfather: Part II
6 . Mickey Blue Eyes
7 . August Rush
8 . Leaving Las Vegas
9 . Machete
10 . Dracula
11 . The Conversation
12 . Superman
13 . West Side Story
14 . American Graffiti
15 . The Score
16 . Peggy Sue Got Married
17 . Insomnia
18 . Love Actually
19 . This Thing of Ours
20 . The Son of No One
21 . The Cotton Club
22 . Donnie Brasco
23 . The Pursuit of D.B. Cooper
24 . Scarface
25 . The Rainmaker
26 . No Reservations
27 . A Streetcar Named Desire
28 . Whatever Works
29 . Signed, Sealed, Delivered
